In [ ]:
import os
import shutil
import subprocess
import pathlib
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from PIL import Image, ImageDraw, ImageFont

# --- 配置区 ---
IMAGE_FOLDER = r"C:\Users\liyil\Desktop\combined_ims"
OUTPUT_DIR   = r"C:\Users\liyil\Desktop\video_output"
FFMPEG_EXE   = r"C:\Users\liyil\Downloads\ffmpeg-master-latest-win64-gpl-shared\bin\ffmpeg.exe"

FPS              = 8
SEGMENT_DURATION = 5  # HLS ts 时长（秒）

RESOLUTIONS = [
    ("360p",  640,  360,  500000),
    ("720p", 1280,  720, 1500000),
    ("1080p",1920, 1080, 3000000),
    ("1440p",2560, 1440, 6000000),
    ("4k",   3840, 2160,10000000),
]

TIME_RANGES = [
    ("06:00:00", "12:00:00"),
    ("12:00:00", "18:00:00"),
]

# --- 工具函数 ---

def parse_image_time(fname):
    try:
        return datetime.strptime(fname.replace(".jpeg", ""), "%Y-%m-%d-%H-%M-%S")
    except:
        return None

def filter_images_by_time(files, t0, t1):
    picked = []
    for fn in files:
        ts = parse_image_time(fn)
        if not ts: continue
        tt = ts.time()
        if (t0 <= tt < t1) or (t0 > t1 and (tt >= t0 or tt < t1)):
            picked.append((ts, fn))
    return sorted(picked)

def stamp_and_copy_frames(sel, src_dir, dst_dir):
    os.makedirs(dst_dir, exist_ok=True)
    font = ImageFont.truetype("C:/Windows/Fonts/arial.ttf", 40)
    for i, (ts, fn) in enumerate(sel, start=1):
        img = Image.open(os.path.join(src_dir, fn)).convert("RGB")
        draw = ImageDraw.Draw(img)
        draw.text((10,10), ts.strftime("%Y-%m-%d %H:%M:%S"), font=font, fill="white")
        img.save(os.path.join(dst_dir, f"{i:05d}.jpeg"))

def process_resolution(key, w, h, frames_dir, out_dir, fps):
    os.makedirs(out_dir, exist_ok=True)
    m3u8   = os.path.join(out_dir, f"{key}.m3u8")
    tpat   = os.path.join(out_dir, f"{key}_%03d.ts")
    bitrate= f"{[b for k,_,_,b in RESOLUTIONS if k==key][0]//1000}k"

    cmd = [
        FFMPEG_EXE,
        '-y','-framerate',str(fps),'-start_number','1',
        '-i', os.path.join(frames_dir, '%05d.jpeg'),
        '-vf', f"scale={w}:{h}", '-c:v','libx264','-preset','veryfast',
        '-pix_fmt','yuv420p','-g',str(fps*2),'-keyint_min',str(fps*2),
        '-sc_threshold','0','-b:v',bitrate,'-maxrate',bitrate,
        '-bufsize','2000k','-an','-f','hls',
        '-hls_time',str(SEGMENT_DURATION),'-hls_list_size','0',
        '-hls_segment_filename', tpat,
        m3u8
    ]
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(f"✅ [{key}] HLS 生成成功")
    except subprocess.CalledProcessError as e:
        print(f"❌ [{key}] FFmpeg 错误（{e.returncode}）：\n{e.stderr}")

def create_master_playlist(dest, resolutions):
    txt = "#EXTM3U\n#EXT-X-VERSION:3\n#EXT-X-INDEPENDENT-SEGMENTS\n"
    for key, w, h, bw in resolutions:
        txt += f"#EXT-X-STREAM-INF:BANDWIDTH={bw},RESOLUTION={w}x{h}\n{key}/{key}.m3u8\n"
    with open(os.path.join(dest, "master.m3u8"), 'w', encoding='utf8') as f:
        f.write(txt)

def create_player_html(base, folders):
    tpl = """<!DOCTYPE html>
<html lang="zh"><head><meta charset="utf-8"><title>HLS 播放</title>
<script src="https://cdn.jsdelivr.net/npm/hls.js@latest"></script>
<style>body{{background:#111;color:#fff;text-align:center;padding:20px;}}select{{margin:5px;}}</style>
</head><body>
<h2>多分辨率 HLS 播放</h2>
时间段：<select id="ts">{} </select>
分辨率：<select id="qs">
  <option value="360p/360p.m3u8">360p</option>
  <option value="720p/720p.m3u8">720p</option>
  <option value="1080p/1080p.m3u8">1080p</option>
  <option value="1440p/1440p.m3u8">1440p</option>
  <option value="4k/4k.m3u8">4k</option>
</select><br><video id="v" controls width="960"></video>
<script>
let hls=null;
function play(){{
  const t=document.getElementById('ts').value;
  const q=document.getElementById('qs').value;
  const url=`${{t}}/${{q}}`;
  if(hls) hls.destroy();
  if(Hls.isSupported()){{
    hls=new Hls();hls.loadSource(url);hls.attachMedia(v);
    hls.on(Hls.Events.MANIFEST_PARSED,()=>v.play());
  }}else{{v.src=url;}}
}}
document.getElementById('ts').onchange=play;
document.getElementById('qs').onchange=play;
play();
</script>
</body></html>"""
    opts = "\n".join(f'<option value="{f}">{f}</option>' for f in folders)
    with open(os.path.join(base,"player.html"),'w',encoding='utf8') as f:
        f.write(tpl.format(opts))

# --- 主流程 ---

def process_all():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    files = [f for f in os.listdir(IMAGE_FOLDER) if f.endswith('.jpeg')]
    folders = []

    for start, end in TIME_RANGES:
        t0 = datetime.strptime(start, "%H:%M:%S").time()
        t1 = datetime.strptime(end,   "%H:%M:%S").time()
        sel = filter_images_by_time(files, t0, t1)
        if not sel: continue

        name = f"{sel[0][0].strftime('%Y-%m-%d_%H-%M')}_to_{sel[-1][0].strftime('%H-%M')}"
        out  = os.path.join(OUTPUT_DIR, name)
        os.makedirs(out, exist_ok=True)
        folders.append(name)

        frm = os.path.join(out, "frames")
        if os.path.exists(frm):
            shutil.rmtree(frm)
        stamp_and_copy_frames(sel, IMAGE_FOLDER, frm)

        with ThreadPoolExecutor() as ex:
            for key,w,h,_ in RESOLUTIONS:
                ex.submit(process_resolution, key, w, h, frm, os.path.join(out,key), FPS)
        create_master_playlist(out, RESOLUTIONS)
        shutil.rmtree(frm)

    create_player_html(OUTPUT_DIR, folders)
    print(" HLS 全部生成完毕！")

    #  每个时间段、每个分辨率生成一个 MP4
    for name in folders:
        base_dir = os.path.join(OUTPUT_DIR, name)

        for key, _, _, _ in RESOLUTIONS:
            m3u8_path = os.path.join(base_dir, key, f"{key}.m3u8")
            mp4_path  = os.path.join(base_dir, f"{key}.mp4")

            if not os.path.exists(m3u8_path):
                print(f"⚠️ 跳过缺失的 {m3u8_path}")
                continue

            cmd = [
                FFMPEG_EXE,
                "-y",
                "-i", m3u8_path,
                "-c", "copy",
                "-bsf:a", "aac_adtstoasc",
                mp4_path
            ]

            try:
                subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                print(f" [{name}] [{key}] MP4 已生成：{mp4_path}")
            except subprocess.CalledProcessError as e:
                print(f"❌ [{name}] [{key}] MP4 合并失败：\n{e.stderr}")

if __name__ == "__main__":
    process_all()



✅ [360p] HLS 生成成功
✅ [720p] HLS 生成成功
✅ [1080p] HLS 生成成功
✅ [1440p] HLS 生成成功
✅ [4k] HLS 生成成功
✅ [720p] HLS 生成成功
✅ [360p] HLS 生成成功
✅ [1080p] HLS 生成成功✅ [1440p] HLS 生成成功

✅ [4k] HLS 生成成功
✅ HLS 全部生成完毕！
🎬 [2025-03-12_06-00_to_11-55] [360p] MP4 已生成：C:\Users\liyil\Desktop\video_output\2025-03-12_06-00_to_11-55\360p.mp4
🎬 [2025-03-12_06-00_to_11-55] [720p] MP4 已生成：C:\Users\liyil\Desktop\video_output\2025-03-12_06-00_to_11-55\720p.mp4
🎬 [2025-03-12_06-00_to_11-55] [1080p] MP4 已生成：C:\Users\liyil\Desktop\video_output\2025-03-12_06-00_to_11-55\1080p.mp4
🎬 [2025-03-12_06-00_to_11-55] [1440p] MP4 已生成：C:\Users\liyil\Desktop\video_output\2025-03-12_06-00_to_11-55\1440p.mp4
🎬 [2025-03-12_06-00_to_11-55] [4k] MP4 已生成：C:\Users\liyil\Desktop\video_output\2025-03-12_06-00_to_11-55\4k.mp4
🎬 [2025-03-12_12-00_to_17-55] [360p] MP4 已生成：C:\Users\liyil\Desktop\video_output\2025-03-12_12-00_to_17-55\360p.mp4
🎬 [2025-03-12_12-00_to_17-55] [720p] MP4 已生成：C:\Users\liyil\Desktop\video_output\2025-03-12_12-00_to_17-55\720

: 